# Azure AI Search Custom Skills Example

This Python notebook demonstrates various powerful scenarios that customers can use in Azure AI Studio leveraging [custom web api skills](https://learn.microsoft.com/en-us/azure/search/cognitive-search-custom-skill-web-api) to search through multi-modal content.

In this demo, we dip our toes into tackling the multi-modal content understanding scenario, by reducing the image + text modality (in rich text + image documents like PPTs, PDFs) into a text-only modality via verbalization.

### Prerequisites
An Azure subscription, with access to Azure OpenAI.

Azure AI Search, any tier, but we recommend Basic or higher for this workload. 

A deployment of the text-embedding-3-small model on Azure OpenAI.

Azure Blob Storage. This notebook connects to your storage account and loads a container with the sample PDFs and images.

Deployed Azure functions to carry out the custom inference tasks with the various deployed language models used in this demo (gpt-4o, gpt-4o-mini, and phi)

In [ ]:
print("Hello World")